In [39]:
from qiskit import *
import numpy as np

In [40]:
def dj_alg(f, N):
    #Building unitary f
    inputs = list(range(2**(N)))
    outputs = f(inputs)
    operator = np.zeros((2**(N+1), 2**(N+1)))
    for i, j in zip(inputs, outputs):
        operator[(2**N)+i, ((((j^1)*2)**N)+i)]=1
        operator[i, (((j*2)**N)+i)]=1
    qc = QuantumCircuit(N+1, N)
    #Building Deutsch Jozsa circuit
    qc.x(N)
    qc.h(range(N+1))
    qc.unitary(operator, range(N+1))
    qc.h(range(N))
    qc.measure(range(N), range(N))
    print(qc)
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc, backend=backend, shots=1)
    job_result = job.result()
    return list(job_result.get_counts())[0]

In [41]:
def bv_prob(a, N): #a = Bernstein Vazirani function paramater as an integer or binary string, N= number of qubits
    if type(a)!=str:
        a_bin = format(a, '0%sb' %N) #Converting a to binary if necessary
    else:
        a_bin = a
    def bv(x): #Bernstein Vazirani function on list of inputs, x = list of inputs of length 2**N
            return [(sum([int(j)*int(k) for (j, k) in zip(list(format(i, '0%sb' %N)), list(a_bin))])%2) for i in x]
    return "a = " +dj_alg(bv, N) #Running Deutsch Jozsa algorithm

In [42]:
bv_prob('11101', 5)

     ┌───┐     ┌──────────┐┌───┐┌─┐            
q_0: ┤ H ├─────┤0         ├┤ H ├┤M├────────────
     ├───┤     │          │├───┤└╥┘┌─┐         
q_1: ┤ H ├─────┤1         ├┤ H ├─╫─┤M├─────────
     ├───┤     │          │├───┤ ║ └╥┘┌─┐      
q_2: ┤ H ├─────┤2         ├┤ H ├─╫──╫─┤M├──────
     ├───┤     │  Unitary │├───┤ ║  ║ └╥┘┌─┐   
q_3: ┤ H ├─────┤3         ├┤ H ├─╫──╫──╫─┤M├───
     ├───┤     │          │├───┤ ║  ║  ║ └╥┘┌─┐
q_4: ┤ H ├─────┤4         ├┤ H ├─╫──╫──╫──╫─┤M├
     ├───┤┌───┐│          │└───┘ ║  ║  ║  ║ └╥┘
q_5: ┤ X ├┤ H ├┤5         ├──────╫──╫──╫──╫──╫─
     └───┘└───┘└──────────┘      ║  ║  ║  ║  ║ 
c: 5/════════════════════════════╩══╩══╩══╩══╩═
                                 0  1  2  3  4 


'a = 11101'

In [43]:
def dj_prob(a, N): #a=c or b, c=constant, b=balanced, N=number of qubits
    def dj(x): #Detusch Jozsa function on list of inputs of length 2**N
        if a=="c":
            return (2**N)*[np.random.choice([0,1])]
        else:
            temp = (2**(N-1))*[0]+(2**(N-1))*[1]
            np.random.shuffle(temp)
            return temp
    result = dj_alg(dj, N) #Running Deutsch Jozsa algorithm
    if int(result, 2)==0: #Interpreting measurement
        return "Constant"
    else:
        return "Balanced"

In [34]:
dj_prob("c", 4)

     ┌───┐     ┌──────────┐┌───┐┌─┐         
q_0: ┤ H ├─────┤0         ├┤ H ├┤M├─────────
     ├───┤     │          │├───┤└╥┘┌─┐      
q_1: ┤ H ├─────┤1         ├┤ H ├─╫─┤M├──────
     ├───┤     │          │├───┤ ║ └╥┘┌─┐   
q_2: ┤ H ├─────┤2 Unitary ├┤ H ├─╫──╫─┤M├───
     ├───┤     │          │├───┤ ║  ║ └╥┘┌─┐
q_3: ┤ H ├─────┤3         ├┤ H ├─╫──╫──╫─┤M├
     ├───┤┌───┐│          │└───┘ ║  ║  ║ └╥┘
q_4: ┤ X ├┤ H ├┤4         ├──────╫──╫──╫──╫─
     └───┘└───┘└──────────┘      ║  ║  ║  ║ 
c: 4/════════════════════════════╩══╩══╩══╩═
                                 0  1  2  3 


'Constant'